In [38]:
for i in range(0, 101):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [72]:
import pandas as pd
import torch

class ParticleDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, detections_file, transform=None):
        self.data_dir = data_dir
        self.detections = pd.read_csv(detections_file, header = None, names=[i for i in range(1,100)]+['label'])
        self.transform = transform

    def __len__(self):
        return len(self.detections)
    
    def __getitem__(self, idx):
        return self.detections.iloc[idx]

In [73]:
data = ParticleDataset('.', 'dataset.csv', transform=None)

In [74]:
data.__len__()

137325

AttributeError: module 'torch' has no attribute 'Subset'